In [4]:
import pandas as pd
import numpy as np
import re
from string import punctuation
from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import train_test_split
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
import twitter

In [5]:
data_set = pd.read_csv("processedCorpus.csv", names=["Tweet", "Sentiment"])

X = data_set.iloc[:, :-1]
y = data_set.iloc[:, -1]

In [6]:
encoder = LabelEncoder()
y = encoder.fit_transform(y)

In [7]:
corpus = list(X["Tweet"]) 

In [8]:
cv = CountVectorizer(max_features=1000)

In [9]:
X = cv.fit_transform(corpus).toarray()

sc = StandardScaler()
X = sc.fit_transform(X)

C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [10]:

X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [11]:
SVMClassifier=SVC()

SVMClassifier.fit(X_train, Y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [12]:

Y_pred = SVMClassifier.predict(X_test)

In [57]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_test, Y_pred) 

0.6852689793866265

In [58]:
from sklearn.metrics import confusion_matrix
confusion_matrix(Y_test, Y_pred)

array([[659, 324],
       [302, 704]], dtype=int64)

In [92]:
print("Predict The sentiment")
data = input("Enter your data to get the sentiment: ")
data = [data]

array = cv.transform(data).toarray()

r = SVMClassifier.predict(array)
print(r)
print data

Predict The sentiment
Enter your data to get the sentiment: 'so unhappy hate'
[1]
['so unhappy hate']


In [69]:
api = twitter.Api(consumer_key='6b9ebwaNU4DDa9G9xF1FhrZQt',
                 consumer_secret='2prgddykMj2b7b9zTeN78BBhdrgdaNxjtSyyoo8iNRzKAZhzMX',
                 access_token_key='817521154847969280-bc6J796tc0cRjlhigiRZIoQVIzeW2Hf',
                 access_token_secret='WAZy2gZ9Ok8NdP3W8TOMNliSUUGrLjesudvqA3nEEh9wH')

In [74]:
def createTestData(search_string):
    try:
        tweets_fetched=api.GetSearch(search_string, count=500)
        
        print "num tweets: "+str(len(tweets_fetched))+" term: "+search_string
        
        return [status.text for status in tweets_fetched]
    except:
        print "Error"
        return None
    
search_string=input("Search for: ")
testData=createTestData(search_string)

Search for: '@realDonaldTrump'
num tweets: 100 term: @realDonaldTrump


In [86]:
class PreProcessTweets:
    def __init__(self):
        self._stopwords=set(stopwords.words('english')+list(punctuation)+['rt', "'s", 'i'])
        
    def processTweets(self, list_of_tweets):
        
        processedTweets=[]
        
        for tweet in list_of_tweets:
            processedTweet=self._processTweet(tweet)
            if len(processedTweet) > 0:
                processedTweets.append(processedTweet)  
        return processedTweets
    
    def _processTweet(self, tweet):
        
        tweet=tweet.lower()
        
        tweet=re.sub('https?://[^\s]+','',tweet)
        
        tweet=re.sub(r'@[^\s]+','',tweet)
        
        tweet=re.sub(r'#([^\s]+)',r'\1',tweet)
        
        tweet=re.sub("[^a-z]", " ",tweet)
        
        tweet=word_tokenize(tweet)
        
        stemmer=PorterStemmer()
        tweet=[stemmer.stem(word) for word in tweet]
        stripper = lambda word: word.strip()
        tweet = list(map(stripper, tweet))
        tweet = filter(None, tweet)
       
        tweet=[word for word in tweet if word not in self._stopwords]
        tweet= " ".join(tweet)
        return tweet
        

In [87]:
tweetProcessor=PreProcessTweets()
ppTestData=tweetProcessor.processTweets(testData)  

In [88]:
print ppTestData[:10]

[u'play polit game believ get make rule consti', u'founder creat govern peopl king let w', u'stun approv among gop voter highest sinc wwii except gwb post trump day coup', u'play polit game believ get make rule constitut', u'lol littl desper benedictdonni project hi treason upon muelleriscom end nigh traitor', u'last person speak transpar tax return', u'con man egomaniac treason innoc man pardon themselv said', u'take long inspector gener report crook hillari slipperi jame comey numer delay', u'take long inspector gener report crook hillari slipperi jame comey numer delay', u'take long inspector gener report crook hillari slipperi jame comey numer delay']


In [111]:
ResultLabels=[]
for tweet in ppTestData:
    Features=cv.transform([tweet]).toarray()
    ResultLabels.append(SVMClassifier.predict(Features)[0])
    

if ResultLabels.count(1)>ResultLabels.count(0):
    print "Result Positive Sentiment: " + str(100*ResultLabels.count(1)/len(ResultLabels))
else:
     print "Result Negative Sentiment: " + str(100*ResultLabels.count(0)/len(ResultLabels))

Result Positive Sentiment: 95
